In [1]:
import os
import sys
import tqdm
import torch
import datetime

import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

from pyntcloud import PyntCloud
from tensorboardX import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Architecture of Encoder

In [3]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        conv1 = [nn.Conv1d(3, 64, kernel_size=1), 
                nn.BatchNorm1d(64),
                nn.ReLU()]
        conv2 = [nn.Conv1d(64, 128, kernel_size=1), 
                nn.BatchNorm1d(128),
                nn.ReLU()]
        conv3 = [nn.Conv1d(128, 256, kernel_size=1), 
                nn.BatchNorm1d(256),
                nn.ReLU()]
        conv4 = [nn.Conv1d(256, 128, kernel_size=1), 
                nn.BatchNorm1d(128),
                nn.AdaptiveMaxPool1d(1)]
        self.conv1 = nn.Sequential(*conv1)
        self.conv2 = nn.Sequential(*conv2)        
        self.conv3 = nn.Sequential(*conv3)
        self.conv4 = nn.Sequential(*conv4)
        
    def forward(self, x):
        out_1 = self.conv1(x)
        out_2 = self.conv2(out_1)
        out_3 = self.conv3(out_2)
        out_4 = self.conv4(out_3)
        print(out_4.shape)
        out_4 = out_4.view(-1, out_4.shape[1])
        print(out_4.shape)
        return out_4

## Architecture of Decoder

In [4]:
class Decoder(nn.Module):
    def __init__(self, num_points):
        super(Decoder, self).__init__()
        linear1 = [nn.Linear(128, 256), 
                nn.BatchNorm1d(256),
                nn.ReLU()]
        linear2 = [nn.Linear(256, 256), 
                nn.BatchNorm1d(256),
                nn.ReLU()]
        linear3 = [nn.Linear(256, 6144), 
                nn.ReLU()]
        self.linear1 = nn.Sequential(*linear1)
        self.linear2 = nn.Sequential(*linear2)
        self.linear3 = nn.Sequential(*linear3)
        self.num_points = num_points
        
    def forward(self, x):
        out_1 = self.linear1(x)
        out_2 = self.linear2(out_1)
        out_3 = self.linear3(out_2)
        
        return out_3.view(-1, 3, self.num_points)

## Architecture of autoencoder

In [5]:
class AutoEncoder(nn.Module):
    def __init__(self, num_points):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder(num_points)
        
    def forward(self, x):
        gfv = self.encoder(x)
        out = self.decoder(gfv)
        
        return out, gfv

## Chamfer Loss

###### For computing the distance matrix, we referred to the following link: https://discuss.pytorch.org/t/efficient-distance-matrix-computation/9065

In [6]:
class ChamferLoss(nn.Module):
    def __init__(self, num_points):
        super(ChamferLoss, self).__init__()
        self.num_points = num_points
        self.loss = torch.FloatTensor([0]).to(device)

        
    def forward(self, predict_pc, gt_pc):
        z, _ = torch.min(torch.norm(gt_pc.unsqueeze(-2) - predict_pc.unsqueeze(-1),dim=1), dim=-2)
            # self.loss += z.sum()
        self.loss = z.sum() / (len(gt_pc)*self.num_points)

        z_2, _ = torch.min(torch.norm(predict_pc.unsqueeze(-2) - gt_pc.unsqueeze(-1),dim=1), dim=-2)
        self.loss += z_2.sum() / (len(gt_pc)*self.num_points)
        return self.loss

In [7]:
autoencoder = AutoEncoder(2048).to(device)
chamfer_loss = ChamferLoss(2048).to(device)

## Dataloader

In [8]:
DATA_DIR = '../latent_3d_points/data/shape_net_core_uniform_samples_2048/'

list_point_clouds = np.load('./list_point_subset.npy')
list_point_clouds = list_point_clouds[:5000]
np.save('list_point_subset_2.npy', list_point_clouds)

X_train, X_test, _, _ = train_test_split(list_point_clouds, list_point_clouds, test_size=0.1, random_state=42)
print(len(X_train))

4500


In [9]:
class PointcloudDatasetAE(Dataset):
    def __init__(self, root, list_point_clouds):
        self.root = root
        self.list_files = list_point_clouds
        
    def __len__(self):
        return len(self.list_files)

    def __getitem__(self, index):
        points = PyntCloud.from_file(self.list_files[index])
        points = np.array(points.points)
        points_normalized = (points - (-0.5)) / (0.5 - (-0.5))
        points = points_normalized.astype(np.float)
        points = torch.from_numpy(points)
        
        return points

In [10]:
train_dataset = PointcloudDatasetAE(DATA_DIR, X_train)
train_dataloader = DataLoader(train_dataset, num_workers=2, shuffle=False, batch_size=48)

test_dataset = PointcloudDatasetAE(DATA_DIR, X_test)
test_dataloader = DataLoader(test_dataset, num_workers=2, shuffle=False, batch_size=1)

for i, data in enumerate(train_dataloader):
    data = data.permute([0,2,1])
    print(data.shape)
    break

torch.Size([48, 3, 2048])


## Optimizer

In [11]:
lr = 1.0e-4
momentum = 0.95
optimizer_AE = torch.optim.Adam(autoencoder.parameters(), lr=lr, betas=(momentum, 0.999))

## Summary Writer and Training Code

In [12]:
ROOT_DIR = './ae_out_copy/'
now =   str(datetime.datetime.now())

if not os.path.exists(ROOT_DIR):
    os.makedirs(ROOT_DIR)

if not os.path.exists(ROOT_DIR + now):
    os.makedirs(ROOT_DIR + now)

LOG_DIR = ROOT_DIR + now + '/logs/'
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

OUTPUTS_DIR = ROOT_DIR  + now + '/outputs/'
if not os.path.exists(OUTPUTS_DIR):
    os.makedirs(OUTPUTS_DIR)

MODEL_DIR = ROOT_DIR + now + '/models/'
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

summary_writer = SummaryWriter(LOG_DIR)

In [13]:
print('Training')
for epoch in range(1000):
    autoencoder.train()
    for i, data in enumerate(train_dataloader):
        data = data.permute([0,2,1]).float().to(device)

        optimizer_AE.zero_grad()
        out_data, gfv = autoencoder(data)

        loss = chamfer_loss(out_data, data)
        loss.backward()
        optimizer_AE.step()
        
        print('Epoch: {}, Iteration: {}, Content Loss: {}'.format(epoch, i, loss.item()))
        summary_writer.add_scalar('Content Loss', loss.item())
#         if i > 2:
#             break
        
#     break
    
    torch.save(autoencoder.state_dict(), MODEL_DIR+'{}_ae_.pt'.format(epoch))

Training
torch.Size([48, 128, 1])
torch.Size([48, 128])
Epoch: 0, Iteration: 0, Content Loss: 0.5484603047370911
torch.Size([48, 128, 1])
torch.Size([48, 128])
Epoch: 0, Iteration: 1, Content Loss: 0.5649220943450928
torch.Size([48, 128, 1])
torch.Size([48, 128])
Epoch: 0, Iteration: 2, Content Loss: 0.5396323800086975
torch.Size([48, 128, 1])
torch.Size([48, 128])
Epoch: 0, Iteration: 3, Content Loss: 0.5203409790992737


## Validation Code

In [14]:
autoencoder = AutoEncoder(2048).to(device)
autoencoder.load_state_dict(torch.load('./ae_out_copy/2019-11-27 01:31:11.870769/models/990_ae_.pt')) # Edit this to add the correct path

<All keys matched successfully>

In [15]:
if not os.path.exists('./eval_output/'):
    os.makedirs('./eval_output/')

In [ ]:
for i in range(X_test.shape[0]):
        points = PyntCloud.from_file(X_test[i])
        points = np.array(points.points)
        points_normalized = (points - (-0.5)) / (0.5 - (-0.5))
        points = points_normalized.astype(np.float)
        points = torch.from_numpy(points).unsqueeze(0)
        points = points.permute([0,2,1]).float().to(device)
        print(points.shape)

        autoencoder.eval()

        with torch.no_grad():
                out_data, gfv = autoencoder(points)
                loss = chamfer_loss(out_data, points)
        print(loss.item())
                
        output = out_data[0,:,:]
        output = output.permute([1,0]).detach().cpu().numpy()

        inputt = points[0,:,:]
        inputt = inputt.permute([1,0]).detach().cpu().numpy()

        fig = plt.figure()
        ax_x = fig.add_subplot(111, projection='3d')
        x_ = output
        ax_x.scatter(x_[:, 0], x_[:, 1], x_[:,2])
        ax_x.set_xlim([0,1])
        ax_x.set_ylim([0,1])
        ax_x.set_zlim([0,1])
        fig.savefig('eval_output/{}_{}.png'.format(i, 'out'))

        fig = plt.figure()
        ax_x = fig.add_subplot(111, projection='3d')
        x_ = inputt
        ax_x.scatter(x_[:, 0], x_[:, 1], x_[:,2])
        ax_x.set_xlim([0,1])
        ax_x.set_ylim([0,1])
        ax_x.set_zlim([0,1])
        fig.savefig('eval_output/{}_{}.png'.format(i, 'in'))

        plt.close('all')

torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.027707599103450775
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.10835500061511993
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.025053545832633972
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.06762520223855972
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03403852880001068
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03191529959440231
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03261226788163185
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.0434688925743103
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03855542838573456
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03399624302983284
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.039982762187719345
torch.Si

torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.030135195702314377
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.06742680072784424
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.04897630959749222
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.029939692467451096
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.02580733224749565
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.052360594272613525
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.02349056303501129
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.043509870767593384
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.043691493570804596
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03224831819534302
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.025501597672700882
torc

torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.032946303486824036
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.04064632207155228
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.030097534880042076
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.024391334503889084
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03119140863418579
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03395456075668335
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.02929896116256714
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.031375035643577576
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.029119238257408142
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.04415946453809738
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.025909777730703354
torc

torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.029833192005753517
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03258246183395386
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03715142607688904
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.05292288959026337
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03636651113629341
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.023071829229593277
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.04046832025051117
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.027169635519385338
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.042372122406959534
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03714899346232414
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03541600704193115
torch.

torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.0320122092962265
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.027795473113656044
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03248195722699165
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.04059755057096481
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.044742293655872345
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.026375245302915573
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.028479885309934616
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.022359011694788933
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.04316185414791107
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.03416859731078148
torch.Size([1, 3, 2048])
torch.Size([1, 128, 1])
torch.Size([1, 128])
0.05349893867969513
torch.